In [1]:
from functions_database_updated import *
import nengo
import nengo_dl

In [2]:
inp = tf.keras.Input(shape=(4))
dense1 = layers.Dense(25, activation=tf.nn.relu, use_bias = True)(inp)
dense2 = layers.Dense(25, activation=tf.nn.relu, use_bias = True)(dense1)
output = layers.Dense(2, use_bias = True)(dense2)
nn_ctlr=tf.keras.Model(inputs=inp,outputs=output)

In [3]:
onnx_model_path = '../controller_double_pendulum_more_robust.onnx'
onnx_model = onnx.load(onnx_model_path)
params_array = []

In [4]:
for initializer in onnx_model.graph.initializer:
    # Convert the initializer tensor to a NumPy array
    tensor_array = onnx.numpy_helper.to_array(initializer)
    params_array.append(tensor_array)

In [5]:
weights_list=[]
bias_list=[]
i=0
for tensor_array in params_array:
    # Convert the initializer tensor to a NumPy array
#     tensor_array = onnx.numpy_helper.to_array(initializer)
    if(i%2==0):
        weights_list.append(tensor_array)
    i = i+1
    

i=0
for tensor_array in params_array:
    # Convert the initializer tensor to a NumPy array
#     tensor_array = onnx.numpy_helper.to_array(initializer)
    if(i%2==1):
        bias_list.append(tensor_array)
    i = i+1

In [6]:
weights_list.reverse()
bias_list.reverse()

In [7]:
for i in range(1, len(weights_list) + 1):
    transposed_weights = weights_list[i-1]
    combined_weights = [transposed_weights, bias_list[i-1]]
    nn_ctlr.layers[i].set_weights(combined_weights)

In [8]:
weights, biases = extract_model_params_tf(nn_ctlr)

In [9]:
def summon_gurobi(dec, eqn, log, to, focus=0):
    all_enc = dec + eqn
    file_path = "Gurobi_encodings_5L_ACC_sim_random.txt"
    with open(file_path, "w") as file:
        for value in all_enc:
            file.write(str(value) + "\n")
#     import time
#     start = time.time()
    model=gp.Model("Encodings")

#     model.Params.MIPGap = GRB.INFINITY
    model.Params.MIPFocus = focus####################################################################
    model.Params.LogToConsole = log
    model.setParam('TimeLimit', to*60*60)
    model.Params.SolutionLimit = 1
#     model.Params.Heuristics=0
#     model.Params.PreCrush=0
#     model.tune()
#     model.Params.Threads = 1

    try:
        f = open(file_path,"r")
        try:
            for l in f:
#                 print(l)
                exec(l)
        finally:
            f.close()
    except IOError:
        pass
#     print('Time taken:', end-start)
#     model.tune()

    model.optimize()
    
    return model

In [10]:
layer_no = 3
neuron_no = 1
time_steps = 5
input_bounds = [[1.0,1.3],[1.0,1.3],[1.0,1.3],[1.0,1.3]]

In [11]:
output_range = [[-5.865708,-6.358360],[-3.692535,-3.416984]]

In [12]:
def can_go_below(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]
#     solver=Solver()
#     solver.reset()
#     solver.set(timeout=240000000)

    # Declarations
    for time in range(1,2):
        for num in range(1,inputs+1):
            declare.append(f"A0_{num}_1 = model.addVar(lb={input_bounds[num-1][0]}, ub={input_bounds[num-1][1]}, name='A0_{num}_1')")

#             declare.append(f"A0_{num}_{time} = model.addVar(name='A0_{num}_{time}')")
#             equations.append(f"model.addConstr(A0_{num}_{time}>={input_bounds[num-1][0]})")
#             equations.append(f"model.addConstr(A0_{num}_{time}<={input_bounds[num-1][1]})")
#             if(time>=2):
#                 equations.append(f"model.addConstr(A0_{num}_{time}==A0_{num}_{time-1})")


    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

#     declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")
#     declare.append(f"P{layer}_{no+1}_0 = model.addVar(name='P{layer}_{no+1}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY,name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
#         declare.append(f"X{layer}_{no}_{time} = model.addVar(name='X{layer}_{no}_{time}')")
#         declare.append(f"P{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY,name='P{layer}_{no}_{time}')")
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no}_{time}')")
#         declare.append(f"q{layer}_{no}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{layer}_{no}_{time}')")
#         declare.append(f"A{layer}_{no}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{layer}_{no}_{time}')")
        declare.append(f"S{layer}_{no+1}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no+1}_{time}')")


        


    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
#     equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")
#     equations.append(f"model.addConstr(P{layer}_{no+1}_0== 0)")

    thresh = 1
    lamb = 1
    M = 99999999
    epsilon = 0.00001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {epsilon})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
    
    out1=f''
    out2=f''
    for time in range(1,time_steps+1):
#         equations.append(f"model.addConstr(P{layer}_{no}_{time} == P{layer}_{no}_{time-1} + S{layer}_{no}_{time})")
        if(time!=1):
            out1+= '+'
            out2+= '+'
        out1+= f'S{layer}_{no}_{time}'
        out2+= f'S{layer}_{no+1}_{time}'
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
        
        equation2 = f'S{layer}_{no+1}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation2 += f' + '
            equation2+=f'(({w[layer-1][no][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation2})+ {b[layer-1][no]})")
    
#     equations.append(f"model.addConstr(P{layer}_{no}_{time_steps}<={cond[0]*time_steps})")
#     equations.append(f'model.setObjective(P{layer}_{no}_{time_steps}, gp.GRB.MINIMIZE)')
#     equations.append(f"model.addConstr({out1}<={cond[0][0]*time_steps})")
#     equations.append(f"model.addConstr({out2}<={cond[0][1]*time_steps})")

    declare.append(f"dec1 = model.addVar(vtype=gp.GRB.BINARY, name='dec1')")
    declare.append(f"dec2 = model.addVar(vtype=gp.GRB.BINARY, name='dec2')")
    
    equations.append(f"model.addConstr({cond[0][0]*time_steps} - ({out1})<= {M}*dec1)")
    equations.append(f"model.addConstr({out1} - ({cond[0][0]*time_steps})<= {M}*(1-dec1))")
    
    equations.append(f"model.addConstr({cond[0][1]*time_steps} - ({out2})<= {M}*dec2)")
    equations.append(f"model.addConstr({out2} - ({cond[0][1]*time_steps})<= {M}*(1-dec2))")
    
    equations.append(f"model.addConstr(dec1 + dec2 >=1)")
    
    
    equations.append(f'model.setObjective({out1}+{out2}, gp.GRB.MINIMIZE)')
#     equations.append(f'model.setObjective(0, gp.GRB.MINIMIZE)')


    return equations, declare ################finall

In [13]:
def can_go_above(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]
#     solver=Solver()
#     solver.reset()
#     solver.set(timeout=240000000)

    # Declarations
    for time in range(1,2):
        for num in range(1,inputs+1):
            declare.append(f"A0_{num}_1 = model.addVar(lb={input_bounds[num-1][0]}, ub={input_bounds[num-1][1]}, name='A0_{num}_1')")

#             declare.append(f"A0_{num}_{time} = model.addVar(name='A0_{num}_{time}')")
#             equations.append(f"model.addConstr(A0_{num}_{time}>={input_bounds[num-1][0]})")
#             equations.append(f"model.addConstr(A0_{num}_{time}<={input_bounds[num-1][1]})")
#             if(time>=2):
#                 equations.append(f"model.addConstr(A0_{num}_{time}==A0_{num}_{time-1})")


    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

#     declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")
#     declare.append(f"P{layer}_{no+1}_0 = model.addVar(name='P{layer}_{no+1}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY,name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
#         declare.append(f"X{layer}_{no}_{time} = model.addVar(name='X{layer}_{no}_{time}')")
#         declare.append(f"P{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY,name='P{layer}_{no}_{time}')")
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no}_{time}')")
#         declare.append(f"q{layer}_{no}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{layer}_{no}_{time}')")
#         declare.append(f"A{layer}_{no}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{layer}_{no}_{time}')")
        declare.append(f"S{layer}_{no+1}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no+1}_{time}')")


        


    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
#     equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")
#     equations.append(f"model.addConstr(P{layer}_{no+1}_0== 0)")

    thresh = 1
    lamb = 1
    M = 99999999
    epsilon = 0.00001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {epsilon})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
    
    out1=f''
    out2=f''
    for time in range(1,time_steps+1):
#         equations.append(f"model.addConstr(P{layer}_{no}_{time} == P{layer}_{no}_{time-1} + S{layer}_{no}_{time})")
        if(time!=1):
            out1+= '+'
            out2+= '+'
        out1+= f'S{layer}_{no}_{time}'
        out2+= f'S{layer}_{no+1}_{time}'
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
        
        equation2 = f'S{layer}_{no+1}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation2 += f' + '
            equation2+=f'(({w[layer-1][no][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation2})+ {b[layer-1][no]})")
    
#     equations.append(f"model.addConstr(P{layer}_{no}_{time_steps}<={cond[0]*time_steps})")
#     equations.append(f'model.setObjective(P{layer}_{no}_{time_steps}, gp.GRB.MINIMIZE)')
#     equations.append(f"model.addConstr({out1}<={cond[0][0]*time_steps})")
#     equations.append(f"model.addConstr({out2}<={cond[0][1]*time_steps})")

    declare.append(f"dec1 = model.addVar(vtype=gp.GRB.BINARY, name='dec1')")
    declare.append(f"dec2 = model.addVar(vtype=gp.GRB.BINARY, name='dec2')")
    
    equations.append(f"model.addConstr({out1} - ({cond[1][0]*time_steps}) <= {M}*dec1)")
    equations.append(f"model.addConstr({cond[1][0]*time_steps} - ({out1}) <= {M}*(1-dec1))")
    
    equations.append(f"model.addConstr({out2} - ({cond[1][1]*time_steps}) <= {M}*dec2)")
    equations.append(f"model.addConstr({cond[1][1]*time_steps} - ({out2}) <= {M}*(1-dec2))")
    
    equations.append(f"model.addConstr(dec1 + dec2 >=1)")
    
    
    equations.append(f'model.setObjective({out1}+{out2}, gp.GRB.MAXIMIZE)')
#     equations.append(f'model.setObjective(0, gp.GRB.MINIMIZE)')


    return equations, declare ################finall

In [14]:
output_range = [[-5.865708,-6.358360],[-3.692535,-3.416984]]

In [15]:
# output_range = [[5.865708,6.358360],[-3.692535,-3.416984]]
delta = 0.0
print('delta',delta)
for tym in range(1,5):

    output_range = [[-5.865708 - delta ,-6.358360 - delta ],[-3.692535 + delta ,-3.416984 + delta]]
    print(output_range)
    print('\nLB Checking')
    avg = 0
    for round in range(1):
        equations, declare = can_go_below(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,0,1.5,0)
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print('Property does not hold')
        elif(model3.status in [9]):
            print('Time out')
        else:
            print(model3.status, 'Property holds')
        print('Runtime',model3.Runtime)
        avg+=model3.Runtime
#     print('\n\nAverage query time',avg/3)

delta 0.0
[[-5.865708, -6.35836], [-3.692535, -3.416984]]

LB Checking
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-20

Time steps 1
4 Property holds
Runtime 0.017999887466430664
[[-5.865708, -6.35836], [-3.692535, -3.416984]]

LB Checking

Time steps 2
4 Property holds
Runtime 0.0
[[-5.865708, -6.35836], [-3.692535, -3.416984]]

LB Checking

Time steps 3
4 Property holds
Runtime 0.051999807357788086
[[-5.865708, -6.35836], [-3.692535, -3.416984]]

LB Checking

Time steps 4
4 Property holds
Runtime 0.15000009536743164


In [16]:
# output_range = [[5.865708,6.358360],[-3.692535,-3.416984]]
delta = 0.0
print('delta',delta)
for tym in range(1,5):

    output_range = [[-5.865708 - delta ,-6.358360 - delta ],[-3.692535 + delta ,-3.416984 + delta]]
    print(output_range)
    print('\nUB Checking')
    avg = 0
    for round in range(1):
        equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
        model3 = summon_gurobi(declare, equations,0,1.5,0)
        print('\nTime steps',tym)
        if(model3.status in [2,10]):
            print('Property does not hold')
        elif(model3.status in [9]):
            print('Time out')
        else:
            print(model3.status, 'Property holds')
        print('Runtime',model3.Runtime)
        avg+=model3.Runtime
#     print('\n\nAverage query time',avg/3)

delta 0.0
[[-5.865708, -6.35836], [-3.692535, -3.416984]]

UB Checking

Time steps 1
Property does not hold
Runtime 0.01100015640258789
[[-5.865708, -6.35836], [-3.692535, -3.416984]]

UB Checking

Time steps 2
Property does not hold
Runtime 0.014999866485595703
[[-5.865708, -6.35836], [-3.692535, -3.416984]]

UB Checking

Time steps 3
Property does not hold
Runtime 0.06299996376037598
[[-5.865708, -6.35836], [-3.692535, -3.416984]]

UB Checking

Time steps 4
4 Property holds
Runtime 0.6129999160766602


# New functions for seperately finding bounds


# Output Neuron 1

In [29]:
def can_go_above(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]
#     solver=Solver()
#     solver.reset()
#     solver.set(timeout=240000000)

    # Declarations
    for time in range(1,2):
        for num in range(1,inputs+1):
            declare.append(f"A0_{num}_1 = model.addVar(lb={input_bounds[num-1][0]}, ub={input_bounds[num-1][1]}, name='A0_{num}_1')")

#             declare.append(f"A0_{num}_{time} = model.addVar(name='A0_{num}_{time}')")
#             equations.append(f"model.addConstr(A0_{num}_{time}>={input_bounds[num-1][0]})")
#             equations.append(f"model.addConstr(A0_{num}_{time}<={input_bounds[num-1][1]})")
#             if(time>=2):
#                 equations.append(f"model.addConstr(A0_{num}_{time}==A0_{num}_{time-1})")


    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

#     declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")
#     declare.append(f"P{layer}_{no+1}_0 = model.addVar(name='P{layer}_{no+1}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY,name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
#         declare.append(f"X{layer}_{no}_{time} = model.addVar(name='X{layer}_{no}_{time}')")
#         declare.append(f"P{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY,name='P{layer}_{no}_{time}')")
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no}_{time}')")
#         declare.append(f"q{layer}_{no}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{layer}_{no}_{time}')")
#         declare.append(f"A{layer}_{no}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{layer}_{no}_{time}')")
        declare.append(f"S{layer}_{no+1}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no+1}_{time}')")


        


    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
#     equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")
#     equations.append(f"model.addConstr(P{layer}_{no+1}_0== 0)")

    thresh = 1
    lamb = 1
    M = 99999999
    epsilon = 0.00001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {epsilon})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
    
    out1=f''
    out2=f''
    for time in range(1,time_steps+1):
#         equations.append(f"model.addConstr(P{layer}_{no}_{time} == P{layer}_{no}_{time-1} + S{layer}_{no}_{time})")
        if(time!=1):
            out1+= '+'
            out2+= '+'
        out1+= f'S{layer}_{no}_{time}'
        out2+= f'S{layer}_{no+1}_{time}'
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
        
#         equation2 = f'S{layer}_{no+1}_{time} == ('
#         for k in range(len(w[layer-1][0])):
#             if(k!=0):
#                 equation2 += f' + '
#             equation2+=f'(({w[layer-1][no][k]}) * A{layer-1}_{k+1}_{time})'
#         equations.append(f"model.addConstr({equation2})+ {b[layer-1][no]})")
    
#     equations.append(f"model.addConstr(P{layer}_{no}_{time_steps}<={cond[0]*time_steps})")
#     equations.append(f'model.setObjective(P{layer}_{no}_{time_steps}, gp.GRB.MINIMIZE)')
#     equations.append(f"model.addConstr({out1}<={cond[0][0]*time_steps})")
#     equations.append(f"model.addConstr({out2}<={cond[0][1]*time_steps})")

    
    equations.append(f"model.addConstr({out1} >= ({cond[0][1]*time_steps}))")
    
    
    equations.append(f'model.setObjective({out1}, gp.GRB.MAXIMIZE)')
#     equations.append(f'model.setObjective(0, gp.GRB.MINIMIZE)')


    return equations, declare ################finall

In [32]:
output_range = [[0,  -3.692535+0.9],[0,0]]
tym = 4
flag = 0
lb =  -3.692535
ub =  -3.692535  + 0.9
checked = []
result = []
while (flag <= 1):
#     mid = (lb+ub)/2
    print("LB:", lb)
    print("UB:", ub)
    mid = random.uniform(lb, ub)
    if(len(checked) > 1 and abs(ub - lb) < 0.001) :
        flag = flag + 1
        
        
    output_range[0][1] = mid
#     print(output_range)
    print("Checking with UB:", output_range[0][1])
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,1,6,0)
    checked.append(output_range[0][1])
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
        lb = mid 
    elif(model3.status in [9]):
        print('Time out')
        break
    else:
        print(model3.status, 'Property holds')
        ub = mid
    result.append(model3.status)
    print('Runtime',model3.Runtime)    
    print(checked)
    print(result)
    print("\n")

LB: -3.692535
UB: -2.792535
Checking with UB: -3.01332848299079
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1655 rows, 1062 columns and 6858 nonzeros
Model fingerprint: 0x984afc57
Variable types: 662 continuous, 400 integer (200 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 1e+08]
Presolve removed 1417 rows and 891 columns
Presolve time: 0.05s
Presolved: 238 rows, 171 columns, 927 nonzeros
Variable types: 90 continuous, 81 integer (39 binary)

Root relaxation: objective -1.034492e+01, 152 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |

     0     0  -12.21869    0   47          -  -12.21869      -     -    0s
     0     0  -12.56497    0   45          -  -12.56497      -     -    0s
     0     0  -12.61368    0   43          -  -12.61368      -     -    0s
     0     0  -12.61675    0   45          -  -12.61675      -     -    0s
     0     0  -12.67828    0   44          -  -12.67828      -     -    0s
     0     0  -13.07168    0   45          -  -13.07168      -     -    0s
     0     0  -13.09305    0   49          -  -13.09305      -     -    0s
     0     0  -13.09305    0   49          -  -13.09305      -     -    0s
     0     0  -13.09305    0   49          -  -13.09305      -     -    0s
     0     0  -13.35418    0   53          -  -13.35418      -     -    0s
     0     0  -13.51352    0   52          -  -13.51352      -     -    0s
     0     0  -13.55625    0   53          -  -13.55625      -     -    0s
     0     0  -13.57092    0   53          -  -13.57092      -     -    0s

Cutting planes:
  Learne

Presolve time: 0.05s
Presolved: 238 rows, 171 columns, 927 nonzeros
Variable types: 90 continuous, 81 integer (39 binary)

Root relaxation: objective -1.034492e+01, 152 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -10.34492    0   44          -  -10.34492      -     -    0s
     0     0  -11.60081    0   38          -  -11.60081      -     -    0s
     0     0  -11.65934    0   42          -  -11.65934      -     -    0s
     0     0  -11.66309    0   43          -  -11.66309      -     -    0s
     0     0  -12.21869    0   47          -  -12.21869      -     -    0s
     0     0  -12.56497    0   45          -  -12.56497      -     -    0s
     0     0  -12.60994    0   43          -  -12.60994      -     -    0s
     0     0  -12.61675    0   45          -  -12.61675      -     -    0s
     0     0  -12.67828    0   44   

     0     0  -13.79604    0   55          -  -13.79604      -     -    0s
     0     0  -14.14930    0   51          -  -14.14930      -     -    0s
     0     0 -1.000e+100    0   55          - infeasible      -     -    0s

Cutting planes:
  Learned: 4
  Gomory: 4
  Cover: 4
  Implied bound: 4
  Clique: 1
  MIR: 39
  StrongCG: 3
  Inf proof: 1
  RLT: 2
  Relax-and-lift: 3

Explored 1 nodes (645 simplex iterations) in 0.46 seconds (0.08 work units)
Thread count was 8 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Time steps 4
4 Property holds
Runtime 0.47699999809265137
[-3.01332848299079, -3.397285195634903, -3.447456213796184, -3.5470616787306595, -3.5491255945092903, -3.5714465796995727, -3.679153048982652, -3.686852635365494, -3.689995985140526]
[4, 4, 4, 4, 4, 4, 4, 4, 4]


LB: -3.692535
UB: -3.689995985140526
Checking with UB: -3.6911934019246107
Set parameter TimeLimi


Root relaxation: objective -1.034492e+01, 152 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -10.34492    0   44          -  -10.34492      -     -    0s
     0     0  -11.60081    0   38          -  -11.60081      -     -    0s
     0     0  -11.65934    0   42          -  -11.65934      -     -    0s
     0     0  -11.66309    0   43          -  -11.66309      -     -    0s
     0     0  -12.21869    0   47          -  -12.21869      -     -    0s
     0     0  -12.56497    0   45          -  -12.56497      -     -    0s
     0     0  -12.61368    0   43          -  -12.61368      -     -    0s
     0     0  -12.61675    0   45          -  -12.61675      -     -    0s
     0     0  -12.67828    0   44          -  -12.67828      -     -    0s
     0     0  -12.85543    0   45          -  -12.85543      -     -    0s
     0  


Cutting planes:
  Learned: 4
  Gomory: 3
  Cover: 5
  Implied bound: 4
  Clique: 1
  MIR: 30
  StrongCG: 3
  Inf proof: 1
  RLT: 2
  Relax-and-lift: 3

Explored 1 nodes (600 simplex iterations) in 0.36 seconds (0.08 work units)
Thread count was 8 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Time steps 4
4 Property holds
Runtime 0.375
[-3.01332848299079, -3.397285195634903, -3.447456213796184, -3.5470616787306595, -3.5491255945092903, -3.5714465796995727, -3.679153048982652, -3.686852635365494, -3.689995985140526, -3.6911934019246107, -3.691431022220961, -3.6915469873596845, -3.6919950634032337, -3.692517587768779]
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]




In [27]:
output_range = [[0,0],[-3.692535,0]]
tym = 2
flag = 0
lb = -3.692535
ub = -3.692535  + 0.9
checked = []
result = []
while (flag <= 1):
#     mid = (lb+ub)/2
    print("LB:", lb)
    print("UB:", ub)
    mid = random.uniform(lb, ub)
    if(len(checked) > 1 and abs(ub - lb) < 0.001) :
        flag = flag + 1
        
        
#     output_range[1][0] = mid
    output_range[1][0] = -3.692535
    
    
#     print(output_range)
    print("Checking with UB:", output_range[1][0])
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,1,6,0)
    checked.append(output_range[1][0])
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
        lb = mid 
    elif(model3.status in [9]):
        print('Time out')
        break
    else:
        print(model3.status, 'Property holds')
        ub = mid
    result.append(model3.status)
    print('Runtime',model3.Runtime)    
    print(checked)
    print(result)
    print("\n")

LB: -3.692535
UB: -2.792535
Checking with UB: -3.692535
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 853 rows, 558 columns and 3454 nonzeros
Model fingerprint: 0xdc0c7b2a
Variable types: 358 continuous, 200 integer (100 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 1e+08]
Presolve removed 704 rows and 463 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Time steps 2
4 Property holds
Runtime 0.0379998683929

Presolve removed 704 rows and 463 columns
Presolve time: 0.02s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Time steps 2
4 Property holds
Runtime 0.026999950408935547
[-3.692535, -3.692535, -3.692535, -3.692535, -3.692535, -3.692535, -3.692535, -3.692535]
[4, 4, 4, 4, 4, 4, 4, 4]


LB: -3.692535
UB: -3.6922315096997327
Checking with UB: -3.692535
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 853 rows, 558 columns and 3454 nonzeros
Model fingerprint: 0xdc0c7b2a
Variable types: 358 continuous, 200 integer (100 binary)
Coefficient statistics:
  Ma

# Output Neuron 2

In [33]:
def can_go_above(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]
#     solver=Solver()
#     solver.reset()
#     solver.set(timeout=240000000)

    # Declarations
    for time in range(1,2):
        for num in range(1,inputs+1):
            declare.append(f"A0_{num}_1 = model.addVar(lb={input_bounds[num-1][0]}, ub={input_bounds[num-1][1]}, name='A0_{num}_1')")

#             declare.append(f"A0_{num}_{time} = model.addVar(name='A0_{num}_{time}')")
#             equations.append(f"model.addConstr(A0_{num}_{time}>={input_bounds[num-1][0]})")
#             equations.append(f"model.addConstr(A0_{num}_{time}<={input_bounds[num-1][1]})")
#             if(time>=2):
#                 equations.append(f"model.addConstr(A0_{num}_{time}==A0_{num}_{time-1})")


    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

#     declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")
#     declare.append(f"P{layer}_{no+1}_0 = model.addVar(name='P{layer}_{no+1}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY,name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
#         declare.append(f"X{layer}_{no}_{time} = model.addVar(name='X{layer}_{no}_{time}')")
#         declare.append(f"P{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY,name='P{layer}_{no}_{time}')")
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no}_{time}')")
#         declare.append(f"q{layer}_{no}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{layer}_{no}_{time}')")
#         declare.append(f"A{layer}_{no}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{layer}_{no}_{time}')")
        declare.append(f"S{layer}_{no+1}_{time} = model.addVar(lb=-GRB.INFINITY, name='S{layer}_{no+1}_{time}')")


        


    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
#     equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")
#     equations.append(f"model.addConstr(P{layer}_{no+1}_0== 0)")

    thresh = 1
    lamb = 1
    M = 99999999
    epsilon = 0.00001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {epsilon})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
    
    out1=f''
    out2=f''
    for time in range(1,time_steps+1):
#         equations.append(f"model.addConstr(P{layer}_{no}_{time} == P{layer}_{no}_{time-1} + S{layer}_{no}_{time})")
        if(time!=1):
            out1+= '+'
            out2+= '+'
        out1+= f'S{layer}_{no}_{time}'
        out2+= f'S{layer}_{no+1}_{time}'
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
        
        equation2 = f'S{layer}_{no+1}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation2 += f' + '
            equation2+=f'(({w[layer-1][no][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation2})+ {b[layer-1][no]})")
    
#     equations.append(f"model.addConstr(P{layer}_{no}_{time_steps}<={cond[0]*time_steps})")
#     equations.append(f'model.setObjective(P{layer}_{no}_{time_steps}, gp.GRB.MINIMIZE)')
#     equations.append(f"model.addConstr({out1}<={cond[0][0]*time_steps})")
#     equations.append(f"model.addConstr({out2}<={cond[0][1]*time_steps})")

    
    equations.append(f"model.addConstr({out2} >= ({cond[1][1]*time_steps}))")
    
    
    equations.append(f'model.setObjective({out2}, gp.GRB.MAXIMIZE)')
#     equations.append(f'model.setObjective(0, gp.GRB.MINIMIZE)')


    return equations, declare ################finall

In [36]:
output_range = [[0,  0],[0,-3.416984]]
tym = 4
flag = 0
lb =  -8.416984
ub =  -3.416984  + 0.9
checked = []
result = []
while (flag <= 1):
#     mid = (lb+ub)/2
    print("LB:", lb)
    print("UB:", ub)
    mid = random.uniform(lb, ub)
    if(len(checked) > 1 and abs(ub - lb) < 0.001) :
        flag = flag + 1
        
        
    output_range[1][1] = mid
#     print(output_range)
    print("Checking with UB:", output_range[1][1])
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,1,6,0)
    checked.append(output_range[1][1])
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
        lb = mid 
    elif(model3.status in [9]):
        print('Time out')
        break
    else:
        print(model3.status, 'Property holds')
        ub = mid
    result.append(model3.status)
    print('Runtime',model3.Runtime)    
    print(checked)
    print(result)
    print("\n")

LB: -8.416984
UB: -2.516984
Checking with UB: -6.283338327392563
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1659 rows, 1062 columns and 6962 nonzeros
Model fingerprint: 0xe0d6109c
Variable types: 662 continuous, 400 integer (200 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 1e+08]
Presolve removed 1422 rows and 891 columns
Presolve time: 0.04s
Presolved: 237 rows, 171 columns, 920 nonzeros
Variable types: 90 continuous, 81 integer (39 binary)
Found heuristic solution: objective -19.3543819

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.03 work units)
Thread count was 8 (of 8 avail

     0     0  -14.48021    0   41          -  -14.48021      -     -    0s
     0     0  -14.48378    0   39          -  -14.48378      -     -    0s
     0     0  -14.48378    0   39          -  -14.48378      -     -    0s

Cutting planes:
  Learned: 2
  Cover: 3
  Implied bound: 7
  MIR: 42
  StrongCG: 7
  Inf proof: 2
  RLT: 4
  Relax-and-lift: 2

Explored 1 nodes (452 simplex iterations) in 0.31 seconds (0.06 work units)
Thread count was 8 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Time steps 4
4 Property holds
Runtime 0.31399989128112793
[-6.283338327392563, -3.5127009145603725, -4.742459408036273, -4.04714603699869, -3.7910036074909716]
[10, 4, 10, 10, 4]


LB: -4.04714603699869
UB: -3.7910036074909716
Checking with UB: -3.9119642593985713
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (

Coefficient statistics:
  Matrix range     [3e-04, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 1e+08]
Presolve removed 1422 rows and 891 columns
Presolve time: 0.05s
Presolved: 237 rows, 171 columns, 920 nonzeros
Variable types: 90 continuous, 81 integer (39 binary)

Root relaxation: objective -1.273806e+01, 115 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -12.73806    0   40          -  -12.73806      -     -    0s
     0     0  -13.70435    0   45          -  -13.70435      -     -    0s
H    0     0                     -15.5422692  -13.70435  11.8%     -    0s

Cutting planes:
  Learned: 3
  Gomory: 2
  Cover: 2
  Implied bound: 3
  MIR: 9
  StrongCG: 2

Explored 1 nodes (195 simplex iterations) in 0.15 seconds (0.04 work units)
Thread count was 8 (of 8 available p


CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1659 rows, 1062 columns and 6962 nonzeros
Model fingerprint: 0x45064088
Variable types: 662 continuous, 400 integer (200 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 1e+08]
Presolve removed 1422 rows and 891 columns
Presolve time: 0.05s
Presolved: 237 rows, 171 columns, 920 nonzeros
Variable types: 90 continuous, 81 integer (39 binary)

Root relaxation: objective -1.273806e+01, 115 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -12.73806    0   40          -  -12.73806      -     -    0s
     0     0  -13.70435    0   45   

Thread count was 8 (of 8 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible or unbounded
Best objective -, best bound -, gap -

Time steps 4
4 Property holds
Runtime 0.39999985694885254
[-6.283338327392563, -3.5127009145603725, -4.742459408036273, -4.04714603699869, -3.7910036074909716, -3.9119642593985713, -3.897854302421156, -3.850986461900618, -3.8941214270069557, -3.8616346977916707, -3.8789284058711284, -3.886784117008141, -3.8790361517059524, -3.8803082736259085]
[10, 4, 10, 10, 4, 10, 10, 4, 10, 4, 4, 10, 4, 4]


LB: -3.886784117008141
UB: -3.8803082736259085
Checking with UB: -3.8834383353989907
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1659 rows, 1

Presolve time: 0.05s
Presolved: 237 rows, 171 columns, 920 nonzeros
Variable types: 90 continuous, 81 integer (39 binary)

Root relaxation: objective -1.273806e+01, 115 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -12.73806    0   40          -  -12.73806      -     -    0s
     0     0  -13.70435    0   45          -  -13.70435      -     -    0s
     0     0  -13.72567    0   45          -  -13.72567      -     -    0s
     0     0  -13.73360    0   49          -  -13.73360      -     -    0s
     0     0  -14.10855    0   35          -  -14.10855      -     -    0s
     0     0  -14.14910    0   35          -  -14.14910      -     -    0s
     0     0  -14.16871    0   43          -  -14.16871      -     -    0s
     0     0  -14.16884    0   43          -  -14.16884      -     -    0s
     0     0  -14.23989    0   42   


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -12.73806    0   40          -  -12.73806      -     -    0s
     0     0  -13.70435    0   45          -  -13.70435      -     -    0s
H    0     0                     -15.5422692  -13.70435  11.8%     -    0s

Cutting planes:
  Learned: 3
  Gomory: 2
  Cover: 2
  Implied bound: 3
  MIR: 9
  StrongCG: 2

Explored 1 nodes (195 simplex iterations) in 0.15 seconds (0.04 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: -15.5423 

Solution limit reached
Best objective -1.554226920009e+01, best bound -1.370434901134e+01, gap 11.8253%

Time steps 4
Property does not hold
Runtime 0.14899992942810059
[-6.283338327392563, -3.5127009145603725, -4.742459408036273, -4.04714603699869, -3.7910036074909716, -3.9119642593985713, -3.897854302421156, -3.850986461900618, -3.8941214270069557, -3.8616346977916707, 

In [38]:
output_range = [[0,  0],[0,-3.416984]]
tym = 3
flag = 0
lb =  -8.416984
ub =  -3.416984  + 0.9
checked = []
result = []
while (flag <= 1):
#     mid = (lb+ub)/2
    print("LB:", lb)
    print("UB:", ub)
    mid = random.uniform(lb, ub)
    if(len(checked) > 1 and abs(ub - lb) < 0.001) :
        flag = flag + 1
        
        
    output_range[1][1] = mid
#     print(output_range)
    print("Checking with UB:", output_range[1][1])
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,1,6,0)
    checked.append(output_range[1][1])
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
        lb = mid 
    elif(model3.status in [9]):
        print('Time out')
        break
    else:
        print(model3.status, 'Property holds')
        ub = mid
    result.append(model3.status)
    print('Runtime',model3.Runtime)    
    print(checked)
    print(result)
    print("\n")

LB: -8.416984
UB: -2.516984
Checking with UB: -7.261138617858074
Set parameter TimeLimit to value 21600
Set parameter SolutionLimit to value 1
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1257 rows, 810 columns and 5234 nonzeros
Model fingerprint: 0x6de81ed0
Variable types: 510 continuous, 300 integer (150 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 1e+08]
Presolve removed 1082 rows and 683 columns
Presolve time: 0.03s
Presolved: 175 rows, 127 columns, 673 nonzeros
Variable types: 63 continuous, 64 integer (27 binary)

Root relaxation: objective -6.536182e+00, 70 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      | 


     0     0   -6.53618    0   30          -   -6.53618      -     -    0s
     0     0   -9.88942    0   24          -   -9.88942      -     -    0s
     0     0  -10.47682    0   24          -  -10.47682      -     -    0s
     0     0  -10.48848    0   24          -  -10.48848      -     -    0s
     0     0  -10.51807    0   21          -  -10.51807      -     -    0s
     0     0  -10.51807    0   21          -  -10.51807      -     -    0s
     0     0  -10.77060    0   28          -  -10.77060      -     -    0s
     0     0  -10.77060    0   27          -  -10.77060      -     -    0s
H    0     0                     -10.8273139  -10.77060  0.52%     -    0s
     0     0  -10.82731    0   30  -10.82731  -10.82731  0.00%     -    0s

Cutting planes:
  Learned: 4
  Gomory: 3
  Cover: 3
  Implied bound: 7
  Clique: 4
  MIR: 21
  StrongCG: 4
  Flow cover: 1
  Network: 1
  RLT: 1
  Relax-and-lift: 6

Explored 1 nodes (232 simplex iterations) in 0.17 seconds (0.03 work units)
Thread

Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1257 rows, 810 columns and 5234 nonzeros
Model fingerprint: 0xe4604b4e
Variable types: 510 continuous, 300 integer (150 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-03, 1e+08]
Presolve removed 1082 rows and 683 columns
Presolve time: 0.04s
Presolved: 175 rows, 127 columns, 673 nonzeros
Variable types: 63 continuous, 64 integer (27 binary)

Root relaxation: objective -6.536182e+00, 70 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -6.53618    0   30          -   -6.53618      -     -    0s
     0     0   -9.88942    0   24          -   -9.88942      -     -    0s
     0     0  -10.47682    0   24          -  -10.47682  

Presolved: 175 rows, 127 columns, 673 nonzeros
Variable types: 63 continuous, 64 integer (27 binary)

Root relaxation: objective -6.536182e+00, 70 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -6.53618    0   30          -   -6.53618      -     -    0s
     0     0   -9.88942    0   24          -   -9.88942      -     -    0s
     0     0  -10.47682    0   24          -  -10.47682      -     -    0s
     0     0  -10.48848    0   24          -  -10.48848      -     -    0s
     0     0  -10.51807    0   21          -  -10.51807      -     -    0s
     0     0  -10.51807    0   18          -  -10.51807      -     -    0s
     0     0  -10.51807    0   18          -  -10.51807      -     -    0s
H    0     0                     -10.8273139  -10.51807  2.86%     -    0s

Cutting planes:
  Learned: 5
  Gomory: 3
  Cover: 1
  Imp

     0     0  -10.51807    0   21          -  -10.51807      -     -    0s
     0     0  -10.51807    0   18          -  -10.51807      -     -    0s
     0     0  -10.51807    0   18          -  -10.51807      -     -    0s
H    0     0                     -10.8273139  -10.82731  0.00%     -    0s
     0     0  -10.82731    0   27  -10.82731  -10.82731  0.00%     -    0s

Cutting planes:
  Learned: 5
  Gomory: 3
  Cover: 1
  Implied bound: 3
  Clique: 1
  MIR: 38
  StrongCG: 5
  Network: 1
  Relax-and-lift: 1

Explored 1 nodes (205 simplex iterations) in 0.17 seconds (0.03 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: -10.8273 
No other solutions better than -10.8273

Optimal solution found (tolerance 1.00e-04)
Best objective -1.082731385529e+01, best bound -1.082731385529e+01, gap 0.0000%

Time steps 3
Property does not hold
Runtime 0.18400001525878906
[-7.261138617858074, -7.1636565675434865, -4.501955797927586, -4.215837903568694, -3.7033258575807246